<a href="https://colab.research.google.com/github/BladeArya/phising-url-detection-ml/blob/main/phisingurls_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phishing Website Detection Feature Extraction(Part -1)**

# **Phising URLs**

## **Data Collection**
The collection of phishing urls is rather easy because of the opensource service called PhishTank. This service provide a set of phishing URLs in multiple formats like csv, json etc. that gets updated hourly. To download the data: https://www.phishtank.com/developer_info.php

In [ ]:
import pandas as pd

In [ ]:
#!wget http://data.phishtank.com/data/online-valid.csv

In [ ]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/BladeArya/phising-url-detection-ml/main/dataset/online-valid.csv')

In [ ]:
raw_data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7403968,https://clientarea.webzworld.com/downloads/osnal,http://www.phishtank.com/phish_detail.php?phis...,2022-01-06T12:06:23+00:00,yes,2022-01-06T12:11:36+00:00,yes,Other
1,7403967,https://pl-olx.426672.site/cash23774872,http://www.phishtank.com/phish_detail.php?phis...,2022-01-06T12:04:48+00:00,yes,2022-01-06T12:11:36+00:00,yes,Other
2,7403966,https://mailserserviceonlinedatedupgradeportal...,http://www.phishtank.com/phish_detail.php?phis...,2022-01-06T12:04:25+00:00,yes,2022-01-06T12:11:36+00:00,yes,Other
3,7403959,https://adobefilestore.weebly.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-01-06T11:57:45+00:00,yes,2022-01-06T12:11:36+00:00,yes,Other
4,7403955,https://httpsloginlive.weebly.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-01-06T11:41:45+00:00,yes,2022-01-06T11:54:18+00:00,yes,Other


Collecting 5,000 Phishing URLs randomly

In [ ]:
phishdataset = raw_data.sample(n = 1000, random_state = 12).copy()
phishdataset = phishdataset.reset_index(drop=True)
phishdataset.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6677520,https://monirshouvo.github.io/fb_responsive/,http://www.phishtank.com/phish_detail.php?phis...,2020-07-15T14:02:13+00:00,yes,2020-07-15T14:08:42+00:00,yes,Other
1,7137320,http://mjaymu1hetizmtl0aa.filesusr.com/html/c6...,http://www.phishtank.com/phish_detail.php?phis...,2021-05-26T12:04:01+00:00,yes,2021-05-26T12:09:02+00:00,yes,Other
2,7390988,https://www.arub-service.org/65VATGHD78UJSIAJS...,http://www.phishtank.com/phish_detail.php?phis...,2021-12-20T13:10:12+00:00,yes,2021-12-20T18:33:26+00:00,yes,Other
3,6870858,https://retraiteenaction.ca/irii/banks/SCO/,http://www.phishtank.com/phish_detail.php?phis...,2020-12-01T14:07:01+00:00,yes,2020-12-01T14:10:43+00:00,yes,Other
4,7329847,https://docs.google.com/forms/d/e/1FAIpQLSeSuq...,http://www.phishtank.com/phish_detail.php?phis...,2021-10-26T01:35:06+00:00,yes,2021-10-27T03:50:16+00:00,yes,Other


Separating Protocol, Domain Name, Address

In [ ]:
seperation_of_protocol = phishdataset['url'].str.split("://",expand = True)

In [ ]:
seperation_of_protocol.head()

,0,1,2,3
0,https,monirshouvo.github.io/fb_responsive/,None,None
1,http,mjaymu1hetizmtl0aa.filesusr.com/html/c69417_30...,None,None
2,https,www.arub-service.org/65VATGHD78UJSIAJS88SCID/4...,None,None
3,https,retraiteenaction.ca/irii/banks/SCO/,None,None
4,https,docs.google.com/forms/d/e/1FAIpQLSeSuqYiwoVF64...,None,None


In [ ]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True) 

In [ ]:
seperation_domain_name.columns=["domain_name","address"] 

In [ ]:
seperation_domain_name.head()

,domain_name,address
0,monirshouvo.github.io,fb_responsive/
1,mjaymu1hetizmtl0aa.filesusr.com,html/c69417_3069841d505568614ed8bca153fc7adf.html
2,www.arub-service.org,65VATGHD78UJSIAJS88SCID/45678788secure97668909...
3,retraiteenaction.ca,irii/banks/SCO/
4,docs.google.com,forms/d/e/1FAIpQLSeSuqYiwoVF64uJl8EwzQPW-vq7_l...


In [ ]:
phishurldata = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)

In [ ]:
phishurldata.columns = ['protocol','domain_name','address']

In [ ]:
phishurldata.head()

,protocol,domain_name,address
0,https,monirshouvo.github.io,fb_responsive/
1,http,mjaymu1hetizmtl0aa.filesusr.com,html/c69417_3069841d505568614ed8bca153fc7adf.html
2,https,www.arub-service.org,65VATGHD78UJSIAJS88SCID/45678788secure97668909...
3,https,retraiteenaction.ca,irii/banks/SCO/
4,https,docs.google.com,forms/d/e/1FAIpQLSeSuqYiwoVF64uJl8EwzQPW-vq7_l...


In [ ]:
phishurldata.shape

(1000, 3)

# **Feature Extraction**
In this step, features are extracted from the URLs dataset.

legitmate-> 0 phishing->1

The extracted features are categorized into


1.   Address Bar based Features
2.   Domain/Abnormal based Features
3.   HTML & Javascript based Features


### **1. Address Bar based Features**


*   IP Address in URL
*   Length of URL
*   Using URL Shortening Services “TinyURL”
*   "@" Symbol in URL
*   Redirection "//" in URL
*   Prefix or Suffix "-" in Domain
*   Depth of URL(Domain and Sub-Domain)
*   "http/https" in Domain name

In [ ]:
import ipaddress
import re

In [ ]:
#1. Using the IP Address
def having_ip_address(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [ ]:
phishurldata['having_ip_address'] = phishdataset['url'].apply(having_ip_address)

In [ ]:
#2. Long URL
def long_url(l):
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [ ]:
phishurldata['long_url'] = phishdataset['url'].apply(long_url) 

In [ ]:
#3. Using URL Shortening Services “TinyURL”
def shortening_service(url):
    match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
    if match:
        return 1
    else:
        return 0

In [ ]:
phishurldata['shortening_service'] = phishdataset['url'].apply(shortening_service)

In [ ]:
#4. URL’s having “@” Symbol
def have_at_symbol(l):
    if "@" in l:
        return 1
    return 0   

In [ ]:
phishurldata['having_@_symbol'] = phishdataset['url'].apply(have_at_symbol)

In [ ]:
#5. Redirecting using “//”
def redirection(l):
    if "//" in l:
        return 1
    return 0

In [ ]:
phishurldata['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [ ]:
#6. Adding Prefix or Suffix Separated by (-) to the Domain
def prefix_suffix_seperation(l):
    if '-' in l:
        return 1
    return 0

In [ ]:
phishurldata['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [ ]:
#7. Sub Domain and Multi Sub Domains
def sub_domains(l):
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [ ]:
phishurldata['sub_domains'] = phishurldata['domain_name'].apply(sub_domains)

In [ ]:
#8. The Existence of “HTTPS” Token in the Domain Part of the URL
def https_token(url):
    match=re.search('https://|http://',url)
    if match.start(0)==0:
        url=url[match.end(0):]
    match=re.search('http|https',url)
    if match:
        return 1
    else:
        return 0

In [ ]:
phishurldata['https_token'] = phishdataset['url'].apply(https_token)

### **2. Domain based Features**


*   Age of Domain
*   DNS Record
*   Website Traffic
*   Domain Registration Length
*   Statical-Report Based Feature

In [ ]:
!pip install python-whois

     |████████████████████████████████| 91 kB 4.3 MB/s 
  Created wheel for python-whois: filename=python_whois-0.7.3-py3-none-any.whl size=87720 sha256=71360a3f207c62ad6b3313a38f638733546b87266bc2bfecbf8d269b965afef1
  Stored in directory: /root/.cache/pip/wheels/11/05/f7/895ce5a73665f77c8274a7d55e34fb3e6b4abbb9a7637e215b
Successfully built python-whois


In [ ]:
import whois
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
from datetime import datetime
import time
import socket
import re

In [ ]:
#9. Age of Domain
def age_of_domain_sub(domain):
    creation_date = domain.creation_date
    expiration_date = domain.expiration_date
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 2
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            return 1
        else:
            return 0

In [ ]:
def age_of_domain_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return age_of_domain_sub(domain_name)

In [ ]:
phishurldata['age_of_domain'] = phishurldata['domain_name'].apply(age_of_domain_main)

Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket


In [ ]:
#10.DNS Record
def dns_record(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
        print(domain_name)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return dns

In [ ]:
phishurldata['dns_record'] = phishurldata['domain_name'].apply(dns_record)

Streaming output truncated to the last 5000 lines.
    "2018-02-01 18:17:51",
    "2016-02-20 13:15:14"
  ],
  "expiration_date": "2022-02-16 23:59:59",
  "name_servers": [
    "ns1.hosting.wildpark.net",
    "ns2.hosting.wildpark.net"
  ]
}
{
  "domain_name": [
    "MACST.CC",
    "macst.cc"
  ],
  "registrar": "GMO INTERNET, INC.",
  "whois_server": "whois.discount-domain.com",
  "referral_url": null,
  "updated_date": [
    "2021-12-15 07:09:06",
    "2021-12-15 16:09:06"
  ],
  "creation_date": "2005-12-28 13:28:31",
  "expiration_date": "2022-12-28 13:28:31",
  "name_servers": [
    "UNS01.LOLIPOP.JP",
    "UNS02.LOLIPOP.JP",
    "uns01.lolipop.jp",
    "uns02.lolipop.jp"
  ],
  "status": [
    "ok https://icann.org/epp#ok",
    "ACTIVE"
  ],
  "emails": [
    "abuse@gmo.jp",
    "kaneko@macst.cc"
  ],
  "dnssec": "unsigned",
  "name": "kouichi kaneko",
  "org": "macst co ltd",
  "address": [
    "4-12-6 komae-shi",
    "etowarukomae1F"
  ],
  "city": "komae-shi",
  "state": "Toky

In [ ]:
# 11. Web traffic 
def web_traffic(url):
  try:
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 2

In [ ]:
phishurldata['web_traffic'] = phishdataset['url'].apply(web_traffic)

In [ ]:
#12. Domain Registration Length
def domain_registration_length_sub(domain):
    expiration_date = domain.expiration_date
    today = time.strftime('%Y-%m-%d')
    today = datetime.strptime(today, '%Y-%m-%d')
    if expiration_date is None:
        return 1
    elif type(expiration_date) is list or type(today) is list :
        return 2             #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
    else:
        registration_length = abs((expiration_date - today).days)
        if registration_length / 365 <= 1:
            return 1
        else:
            return 0
    

In [ ]:
def domain_registration_length_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return domain_registration_length_sub(domain_name)

In [ ]:
phishurldata['domain_registration_length'] = phishurldata['domain_name'].apply(domain_registration_length_main)

Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket


In [ ]:
phishurldata.shape

(1000, 15)

In [ ]:
#13.Statical-Report Based Feature
def statistical_report(url):
    hostname = url
    h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
    z = int(len(h))
    if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
            hostname = hostname[:h[0][0]]
    url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
    try:
        ip_address = socket.gethostbyname(hostname)
        ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
    except:
        return 1

    if url_match:
        return 1
    else:
        return 0


In [ ]:
phishurldata['statistical_report'] = phishdataset['url'].apply(statistical_report)

### **3.HTML and JavaScript based Features**


*   IFrame Redirection
*   Status Bar Customization


In [ ]:
#14.iFrame Redirection
def iframe_sub(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

In [ ]:
def iframe_main(url):
  try:
    response = requests.get(url)
  except:
    response = ''
  
  return iframe_sub(response)

In [ ]:
phishurldata['iframe'] = phishdataset['url'].apply(iframe_main)

In [ ]:
#15. Status Bar Customization 
def mouse_over_sub(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [ ]:
def mouse_over_main(url):
  try:
    response = requests.get(url)
  except:
    response = ''
  
  return mouse_over_sub(response)

In [ ]:
phishurldata['mouse_over'] = phishdataset['url'].apply(mouse_over_main)

In [ ]:
phishurldata['label'] = 1

In [ ]:
phishurldata.shape

(1000, 19)

In [ ]:
phishurldata.head()

,protocol,domain_name,address,having_ip_address,long_url,shortening_service,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,https_token,age_of_domain,dns_record,web_traffic,domain_registration_length,statistical_report,iframe,mouse_over,label
0,https,monirshouvo.github.io,fb_responsive/,0,0,0,0,0,0,0,0,1,0,1,1,0,1,1,1
1,http,mjaymu1hetizmtl0aa.filesusr.com,html/c69417_3069841d505568614ed8bca153fc7adf.html,0,1,0,0,0,0,0,0,2,0,1,2,0,1,1,1
2,https,www.arub-service.org,65VATGHD78UJSIAJS88SCID/45678788secure97668909...,0,1,0,0,0,1,0,0,0,0,1,1,0,1,1,1
3,https,retraiteenaction.ca,irii/banks/SCO/,0,0,0,0,0,0,0,0,0,0,2,1,0,1,1,1
4,https,docs.google.com,forms/d/e/1FAIpQLSeSuqYiwoVF64uJl8EwzQPW-vq7_l...,0,1,0,0,0,0,0,0,2,0,1,2,0,1,1,1


## Storing the extracted legitimate URLs fatures to csv file

In [ ]:
phishurldata.to_csv('extracted_phishing_dataset.csv', index= False)